In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
import cv2
import numpy as np
from tensorflow import keras
from ultralytics import YOLO

In [ ]:
# Load the trained model VGG19 
VGG19_trained_model = keras.models.load_model("../save/VGG19_Model.h5")

# Load a trained YOLOv8 model
YOLOv8_trained_model = YOLO("../save/YoloV8_Face_Detection_Model.pt")

In [ ]:
Total_Prob_AI_pred = 0.0
Total_Prob_Real_pred = 0.0

# Function to upload an image
def upload_image():

    global Total_Prob_AI_pred 
    global Total_Prob_Real_pred 

    Total_Prob_AI_pred = 0.0
    Total_Prob_Real_pred = 0.0
    
    file_path = filedialog.askopenfilename()
    
    if file_path:
        img = Image.open(file_path)
        img.thumbnail((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        label_img.config(image=img_tk)
        label_img.image = img_tk

        # Detect faces
        img_cv = cv2.imread(file_path)
        results = YOLOv8_trained_model(img_cv , conf = 0.70)
     
        # Extract bounding boxes
        boxes = results[0].boxes.xyxy.cpu().tolist()

        cropped_imgs = []

        # Iterate through each bounding box
        for i, box in enumerate(boxes):
    
            x_min, y_min, x_max, y_max = map(int, box)  # Convert coordinates to integers for indexing
        
            # Calculate the width and height of the bounding box
            width = x_max - x_min
            height = y_max - y_min
        
            # Determine the size of the square
            side_length = max(width, height)
        
            # Calculate the center of the bounding box
            center_x = x_min + width // 2
            center_y = y_min + height // 2
        
            # Calculate the new square bounding box coordinates
            new_x_min = center_x - side_length // 2
            new_y_min = center_y - side_length // 2
            new_x_max = center_x + side_length // 2
            new_y_max = center_y + side_length // 2
        
            # Ensure the coordinates are within the image boundaries
            new_x_min = max(0, new_x_min)
            new_y_min = max(0, new_y_min)
            new_x_max = min(img_cv.shape[1], new_x_max)
            new_y_max = min(img_cv.shape[0], new_y_max)
        
            # Crop faces from the source image
            cropped_img = img_cv[new_y_min:new_y_max, new_x_min:new_x_max]
            #print ( new_y_min, new_y_max, new_x_min , new_x_max)
        
            # Save the cropped image 
            cv2.imwrite(f'cropped_img{i}.jpg', cropped_img)  
                
            # Resize the image (VGG19 --- 224x224 input)
            img = cv2.resize(cropped_img,(224,224))
        
            cropped_imgs.append(img)

            # End of for loop
        
        print("No. of faces : ",len(cropped_imgs))   
        
        # Convert to NumPy array and normalize 
        cropped_imgs = np.array(cropped_imgs)
        cropped_imgs = cropped_imgs/255.0

        # predictions
        predictions = VGG19_trained_model.predict(cropped_imgs)

        print("predictions probabilities:\n",predictions)

        i = 0
        for prediction in predictions:
            
            print('\n',prediction)
            print("The probability of the image belonging to the AI-images class : ",prediction[0])
            print("The probability of the image belonging to the Real images class : ",prediction[1])
            print(type(prediction[0]))

            Total_Prob_AI_pred = Total_Prob_AI_pred + prediction[0]
            Total_Prob_Real_pred = Total_Prob_Real_pred + prediction[1]
            
            if prediction[0] > prediction[1]:
                print("lebel/class : 0 ",)
                print("Al-generated image")
            else:
                print("lebel/class : 1 ")
                print("Real image")
            image_path = f'cropped_img{i}.jpg'
            i +=1
            try:
                # Open the image using PIL
                image = Image.open(image_path)
                display(image)
            except FileNotFoundError:
                print(f"Error: Image file '{image_path}' not found.")
    
    label.pack_forget()  # Hide the label
    check_button.pack()  # Show the check button    
            

def button_click():
    
    if Total_Prob_AI_pred > Total_Prob_Real_pred :
        label.config(text="\nAI Generated Image", fg="red", font=("Helvetica", 16, "bold"))
    else:
        label.config(text="\nReal Image", fg="green", font=("Helvetica", 16, "bold"))

    label.pack()


    
# Initialize the Tkinter root window
root = tk.Tk()
root.title("AI-Image Detector")
root.geometry("600x550")

# Add a button to upload images
btn_upload = tk.Button(root, text="Upload Image", command=upload_image)
btn_upload.pack(pady=20)

# Add a label to display the image
label_img = tk.Label(root)
label_img.pack(pady=20)

# Add a check button 
check_button = tk.Button(root, text="Check", command=button_click)
check_button.pack_forget()  # Hide the check button initially

label = tk.Label(root, text="")
label.pack_forget()

# Run the Tkinter main loop
root.mainloop()
